In [17]:
import numpy as np
#import mnist
import keras
from keras.datasets import fashion_mnist

In [18]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

In [19]:
# Normalize the images.
train_images = (train_images / 255) - 0.5
test_images = (test_images / 255) - 0.5

In [20]:
print(train_images.shape) # (60000, 28, 28)
print(test_images.shape)  # (10000, 28, 28)

(60000, 28, 28)
(10000, 28, 28)


In [21]:
# Flatten the images.
train_images = train_images.reshape((-1, train_images.shape[1] * train_images.shape[2]))
test_images = test_images.reshape((-1, test_images.shape[1] * test_images.shape[2]))

print(train_images.shape) # (60000, 784)
print(test_images.shape)  # (10000, 784)

(60000, 784)
(10000, 784)


In [22]:
np.unique(train_labels)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8)

In [23]:
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical
from keras.datasets import mnist

In [24]:
model = Sequential()
model.add(Dense(64, input_shape=(784,), activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(10, activation='sigmoid'))

In [42]:
# Compile the model.
model.compile(
  optimizer='adam',
  loss='categorical_crossentropy',
  metrics=['accuracy'],
)

In [43]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 64)                50240     
_________________________________________________________________
dense_4 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_5 (Dense)              (None, 10)                650       
Total params: 55,050
Trainable params: 55,050
Non-trainable params: 0
_________________________________________________________________


In [48]:
# Train the model.
model.fit(
  train_images,
  to_categorical(train_labels),
  epochs=10,
  batch_size=200,
)

# Evaluate the model.
model.evaluate(
  test_images,
  to_categorical(test_labels)
)

# Predict on the first 5 test images.
predictions = model.predict(test_images)

# Print our model's predictions.
print(np.argmax(predictions, axis=1)[:10]) # [7, 2, 1, 0, 4]

# Check our predictions against the ground truths.
print(test_labels[:10]) # [7, 2, 1, 0, 4]

Epoch 1/10
300/300 [==============================] - 1s 3ms/step - loss: 0.0894 - accuracy: 0.9668
Epoch 2/10
300/300 [==============================] - 1s 3ms/step - loss: 0.0823 - accuracy: 0.9695
Epoch 3/10
300/300 [==============================] - 1s 3ms/step - loss: 0.0828 - accuracy: 0.9697
Epoch 4/10
300/300 [==============================] - 1s 3ms/step - loss: 0.0794 - accuracy: 0.9704
Epoch 5/10
300/300 [==============================] - 1s 4ms/step - loss: 0.0771 - accuracy: 0.9720
Epoch 6/10
300/300 [==============================] - 1s 3ms/step - loss: 0.0800 - accuracy: 0.9711
Epoch 7/10
300/300 [==============================] - 1s 3ms/step - loss: 0.0756 - accuracy: 0.9725
Epoch 8/10
300/300 [==============================] - 1s 3ms/step - loss: 0.0770 - accuracy: 0.9714
Epoch 9/10
300/300 [==============================] - 1s 3ms/step - loss: 0.0767 - accuracy: 0.9720
Epoch 10/10
313/313 [==============================] - 0s 995us/step - loss: 0.5902 - accuracy: 0.87

In [55]:
keras.metrics.binary_accuracy(test_labels, np.argmax(predictions, axis=1))

<tf.Tensor: shape=(), dtype=float32, numpy=0.1798>